In [1]:
from download import *
from databuilder import *
from dataset import *
from processing import *

from torchvision import transforms

df = download_mimii("../../data/mimii", "0_dB")

../../data/mimii/0_dB/fan.zip already exist.
../../data/mimii/0_dB/fan.zip already extracted.
../../data/mimii/0_dB/pump.zip already exist.
../../data/mimii/0_dB/pump.zip already extracted.
../../data/mimii/0_dB/slider.zip already exist.
../../data/mimii/0_dB/slider.zip already extracted.
../../data/mimii/0_dB/valve.zip already exist.
../../data/mimii/0_dB/valve.zip already extracted.


In [2]:
params = {
    "batch_size":32,
    "epoch": 5,
    "learning_rate": 0.0001,
    "path": "./saved/epoch900/"
}

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

In [3]:
# df["label"].value_counts()
df["label"].value_counts().sort_index()

0    14719
1     1475
2      456
3      890
4      479
Name: label, dtype: int64

In [5]:
train_df, val_df, test_df = split_dataframe(df, 0.6, 0.2)
# print(train_df.shape)
# print(train_df)

(18019, 2)
                                                    data  label
0      [0.0020599365, 0.004432678, 0.0065078735, 0.00...      1
1      [-0.00050354004, -0.00019454956, 0.0007286072,...      1
2      [-0.015193939, -0.014381409, -0.013492584, -0....      1
3      [0.012817383, 0.013893127, 0.015125275, 0.0149...      1
4      [-0.010471344, -0.012817383, -0.015136719, -0....      1
...                                                  ...    ...
18014  [0.00061798096, 0.0021476746, 0.002922058, 0.0...      0
18015  [0.0021514893, 0.0017585754, 0.0011749268, 0.0...      0
18016  [0.0040626526, 0.004486084, 0.0033035278, 0.00...      0
18017  [-0.0013008118, -0.0023880005, -0.0028762817, ...      0
18018  [-0.00031280518, 0.00023651123, 0.00049209595,...      0

[18019 rows x 2 columns]


In [6]:
train_data, train_label = build_from_dataframe(train_df, 4096, 4096//2, False)
val_data, val_label = build_from_dataframe(val_df, 4096, 4096//2, False)
test_data, test_label = build_from_dataframe(test_df, 4096, 4096//2, False)
# print(train_data.shape)
# print(train_data)

(810855, 4096)
[[ 2.0599365e-03  4.4326782e-03  6.5078735e-03 ... -1.8882751e-03
  -2.3345947e-03 -3.1280518e-03]
 [ 2.9563904e-03  2.5138855e-03  3.0975342e-03 ...  9.1590881e-03
   7.2135925e-03  7.3356628e-03]
 [-1.4266968e-03 -3.8146973e-06  8.0108643e-05 ... -6.1149597e-03
  -5.3596497e-03 -4.7531128e-03]
 ...
 [-3.4980774e-03 -4.1923523e-03 -4.1732788e-03 ...  2.0599365e-04
   1.0643005e-03  1.0108948e-03]
 [ 4.0817261e-04  2.6969910e-03  4.2304993e-03 ...  5.8746338e-03
   4.3945312e-03  3.4751892e-03]
 [ 5.8364868e-04  3.0899048e-04  3.0899048e-04 ... -4.6539307e-04
  -5.0735474e-04 -5.4550171e-04]]


In [14]:
transform = transforms.Compose([
    melspectogram(),
    ExpandDim(),
    ReSize(3, 32, 32),
    NpToTensor()
])

In [15]:
train_dataset = NumpyDataset(data=train_data, label=train_label, 
                             transform=transform, target_transform=NpToTensor())
val_dataset = NumpyDataset(data=val_data, label=val_label, 
                             transform=transform, target_transform=NpToTensor())
test_dataset = NumpyDataset(data=test_data, label=test_label, 
                             transform=transform, target_transform=NpToTensor())

# print(train_dataset.data)
# print(train_dataset.label)

In [16]:
train_loader = get_dataloader(dataset=train_dataset,
                              batch_size=params["batch_size"],
                              shuffle=True)
val_loader = get_dataloader(dataset=val_dataset,
                              batch_size=params["batch_size"],
                              shuffle=False)
test_loader = get_dataloader(dataset=test_dataset,
                              batch_size=params["batch_size"],
                              shuffle=False)

In [17]:
# # 데이터로더 동작 확인
# for i, (data, target) in enumerate(train_loader):
#     print(i, data.shape, target.shape, data[0].shape)
#     if i > 9:
#         break